## Libraries

In [83]:
import numpy as np
import pulp as pl
import random

## Src

In [84]:
def generate_vrptw_data(
    num_customers,
    max_demand,
    max_travel_time,
    max_service_time,
    max_time_window,
    max_cost,
    random_seed=None,
):
    if random_seed is not None:
        random.seed(random_seed)
        np.random.seed(random_seed)

    demands = np.array(
        [0] + [random.randint(1, max_demand) for _ in range(num_customers)]
    )

    travel_times = np.zeros((num_customers + 1, num_customers + 1), dtype=int)
    for i in range(num_customers + 1):
        for j in range(i + 1, num_customers + 1):
            travel_times[i, j] = travel_times[j, i] = random.randint(1, max_travel_time)

    costs = np.zeros((num_customers + 1, num_customers + 1), dtype=int)
    for i in range(num_customers + 1):
        for j in range(i + 1, num_customers + 1):
            costs[i, j] = costs[j, i] = random.randint(1, max_cost)

    time_windows = np.array(
        [(0, max_time_window)]
        + [
            (
                random.randint(0, max_time_window // 2),
                random.randint(max_time_window // 2, max_time_window),
            )
            for _ in range(num_customers)
        ]
    )

    service_times = np.array(
        [0] + [random.randint(1, max_service_time) for _ in range(num_customers)]
    )

    return demands, travel_times, costs, time_windows, service_times

## Constants

In [85]:
max_demand = 5
max_travel_time = 30
max_service_time = 15
max_time_window = 100
N = 10  # Nombre de points de collecte (sans compter le dépôt)
num_camions = 2
camion_capacity = 15
max_cost = 50

demands, travel_times, costs, time_windows, service_times = generate_vrptw_data(
    N,
    max_demand,
    max_travel_time,
    max_service_time,
    max_time_window,
    max_cost,
    random_seed=42
)
print("Demands:", demands)
print("Travel Times:\n", travel_times)
print("Costs:\n", costs)
print("Time Windows:", time_windows)
print("Service Times:", service_times)

Demands: [0 1 1 3 2 2 2 1 5 1 5]
Travel Times:
 [[ 0 14  2  1  3  7  8 17 20  1 18]
 [14  0  7 23 21 23 18 14  8 15 19]
 [ 2  7  0  9 26 28  1 25 26  6 23]
 [ 1 23  9  0 14 11  9  5  7 25 11]
 [ 3 21 26 14  0  4  3 13  4 12 28]
 [ 7 23 28 11  4  0 12 20  9 26  2]
 [ 8 18  1  9  3 12  0 24 15 18  4]
 [17 14 25  5 13 20 24  0 30 13  3]
 [20  8 26  7  4  9 15 30  0 18 10]
 [ 1 15  6 25 12 26 18 13 18  0 27]
 [18 19 23 11 28  2  4  3 10 27  0]]
Costs:
 [[ 0 41 40 24 37 13 46  5  3 43 15]
 [41  0 50 19  6 15  7 25 18 30 41]
 [40 50  0 24 11 24 23 14 43 18 45]
 [24 19 24  0 44 42  5 39 41 11 35]
 [37  6 11 44  0 47 16 11 30 25 18]
 [13 15 24 42 47  0 41 45 36 15 44]
 [46  7 23  5 16 41  0 21 50 50  4]
 [ 5 25 14 39 11 45 21  0 15  3 21]
 [ 3 18 43 41 30 36 50 15  0 26 18]
 [43 30 18 11 25 15 50  3 26  0  5]
 [15 41 45 35 18 44  4 21 18  5  0]]
Time Windows: [[  0 100]
 [ 13  86]
 [ 45  70]
 [ 13  91]
 [ 31  75]
 [ 41  79]
 [  9  66]
 [  8  65]
 [ 47  85]
 [ 34  66]
 [ 47  87]]
Service Times:

Modèle de minimisation de coût

In [86]:
vrptw_model = pl.LpProblem("Optimisation_Collecte_Dechets", pl.LpMinimize)

Variables de décision

In [87]:
x = pl.LpVariable.dicts(
    "x",
    ((i, j, k) for i in range(N + 1) for j in range(N + 1) for k in range(num_camions)),
    cat="Binary",
)
s = pl.LpVariable.dicts(
    "s",
    ((i, k) for i in range(N + 1) for k in range(num_camions)),
    lowBound=0,
    cat="Continuous",
)

Fonction objectif : minimiser la distance totale parcourue

In [88]:
vrptw_model += (
    pl.lpSum(
        costs[i][j] * x[i, j, k]
        for i in range(N + 1)
        for j in range(N + 1)
        for k in range(num_camions)
    ),
    "TotalCost",
)

## constraints

1. Each customer is visited exactly once

In [89]:
for i in range(1, N + 1):
    vrptw_model += (
        pl.lpSum(
            x[i, j, k] for j in range(N + 1) for k in range(num_camions)
        )
        == 1,
        f"VisitOnce_{i}",
    )

In [90]:
for i in range(N + 1):
    for k in range(num_camions):
        vrptw_model += x[i, i, k] == 0, f"NoSelfLoop_{i}_{k}"


2. Vehicle capacity constraint

In [91]:
for k in range(num_camions):
    vrptw_model += (
        pl.lpSum(
            demands[i] * x[i, j, k]
            for i in range(N + 1)
            for j in range(N + 1)
        )
        <= camion_capacity,
        f"Capacity_{k}",
    )

3. Flow conservation constraint (leave each node after entering it)

In [92]:
for k in range(num_camions):
    for i in range(N + 1):
        vrptw_model += (
            pl.lpSum(x[i, j, k] for j in range(N + 1))
            == pl.lpSum(x[j, i, k] for j in range(N + 1)),
            f"FlowConservation_{i}_{k}",
        )


4. Start and end at the depot

In [93]:
for k in range(num_camions):
    vrptw_model += (
        pl.lpSum(x[0, j, k] for j in range(1, N + 1)) == 1,
        f"StartAtDepot_{k}",
    )
    vrptw_model += (
        pl.lpSum(x[j, 0, k] for j in range(1, N + 1)) == 1,
        f"EndAtDepot_{k}",
    )

5. Time window constraints

In [94]:
for k in range(num_camions):
    for i in range(N + 1):
        for j in range(1, N + 1):  # skip depot as destination
            if i != j:
                vrptw_model += (
                    s[i, k] + travel_times[i][j] + service_times[i] <= s[j, k] + (1 - x[i, j, k]) * 1e6
                ), f"TimeWindow_{i}_{j}_{k}"

    for i in range(1, N + 1):  # skip depot
        vrptw_model += (time_windows[i][0] <= s[i, k]), f"TimeWindowStart_{i}_{k}"
        vrptw_model += (s[i, k] <= time_windows[i][1]), f"TimeWindowEnd_{i}_{k}"


---

## Solve

In [95]:
vrptw_model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/cuphead/Projects/VRPTW-Meta-Heuristique/venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/6cf6a91ab8ab4697b259506722168fa0-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/6cf6a91ab8ab4697b259506722168fa0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 305 COLUMNS
At line 2614 RHS
At line 2915 BOUNDS
At line 3158 ENDATA
Problem MODEL has 300 rows, 264 columns and 1582 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 88 - 0.00 seconds
Cgl0002I 22 variables fixed
Cgl0003I 20 fixed, 0 tightened bounds, 176 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 156 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 162 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 162 strengthened 

1

In [96]:
if vrptw_model.status == pl.LpStatusOptimal:
    print("Optimal solution found with total cost:", pl.value(vrptw_model.objective))
    routes = []
    for k in range(num_camions):
        route = [0]  # Start at depot
        current_location = 0
        while True:
            next_location = None
            for j in range(N + 1):
                if pl.value(x[current_location, j, k]) == 1:
                    route.append(j)
                    next_location = j
                    break
            if next_location == 0 or next_location is None:
                break
            current_location = next_location
        routes.append(route)
    print("Routes:", routes)
else:
    print("No optimal solution found.")

Optimal solution found with total cost: 192.0
Routes: [[0, 7, 9, 2, 6, 10, 5, 0], [0, 3, 1, 4, 8, 0]]


In [97]:
count = 0
for i in range(N):
    count += pl.value(x[i, 2, 0])
    if count == 1: print(i)

print(count)

9
1.0


In [98]:
pl.value(x[2,2,0])

0.0

---